# Demo rep to issue https://github.com/slundberg/shap/issues/1678

In [1]:
import numpy as np
import shap
import sklearn
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import torch

input_size = 16
np.random.seed(0)
torch.random.manual_seed(0)
print(f'shap {shap.__version__}, torch {torch.__version__}, numpy {np.__version__}, sklearn {sklearn.__version__}')

shap 0.37.0, torch 1.8.1, numpy 1.19.2, sklearn 0.24.1


In [2]:
# defining a network using torch.nn.functional gives incorrect shap values 

class FunctionalTestNet(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.layer_1 = torch.nn.Linear(input_size, 32)
        self.layer_2 = torch.nn.Linear(32, 32)
        self.layer_3 = torch.nn.Linear(32, 1)
        self.activation = torch.nn.functional.relu


    def forward(self, x):
        h = self.activation(self.layer_1(x))
        h = self.activation(self.layer_2(h))
        z = self.layer_3(h)
        return z

X, y = make_regression(n_samples=100, n_features=input_size, random_state=0, noise=5.0, bias=25)
X, y = torch.from_numpy(X.astype(np.float32)), torch.from_numpy(y.reshape(-1, 1).astype(np.float32))
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=64)

test_net = FunctionalTestNet()

loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(test_net.parameters(), lr=0.001)

for k in range(5000):
    optimizer.zero_grad()
    y_pred = test_net(X_train)
    loss_value = loss_function(y_pred, y_train)
    loss_value.backward()
    optimizer.step()
    
test_net.eval()
e = shap.DeepExplainer(test_net, X_train)
shap_values = e.shap_values(X_test)

# these values should be the same
test_net(X_test)[:10, 0], e.expected_value + np.sum(shap_values, axis=1)[:10]

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


(tensor([ 184.7158,   60.4674,  -58.9145, -146.7091,  205.9765,   23.7587,
          207.2591,  129.6265,  198.8885,   35.1432], grad_fn=<SelectBackward>),
 array([ 149.08664376,   26.62465567,  -60.81331158, -181.06401011,
         296.0004769 ,   -3.23394275,  157.46819324,  124.32344535,
         206.36434121,  -10.48587956]))

In [3]:
# Also issues if we use torch.nn.Module, but only instantiate once

class ModuleTestNet(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.layer_1 = torch.nn.Linear(input_size, 32)
        self.layer_2 = torch.nn.Linear(32, 32)
        self.layer_3 = torch.nn.Linear(32, 1)
        self.activation = torch.nn.ReLU()


    def forward(self, x):
        h = self.activation(self.layer_1(x))
        h = self.activation(self.layer_2(h))
        z = self.layer_3(h)
        return z
    

test_net = ModuleTestNet()

loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(test_net.parameters(), lr=0.001)

for k in range(5000):
    optimizer.zero_grad()
    y_pred = test_net(X_train)
    loss_value = loss_function(y_pred, y_train)
    loss_value.backward()
    optimizer.step()
    
e = shap.DeepExplainer(test_net, X_train)
shap_values = e.shap_values(X_test)

test_net(X_test)[:10, 0], e.expected_value + np.sum(shap_values, axis=1)[:10]

(tensor([ 184.8368,   22.0020,  -33.4058, -163.9702,  236.3126,   37.6507,
          165.4864,  126.7657,  198.3206,    1.7161], grad_fn=<SelectBackward>),
 array([ 236.2532976 ,   30.13508737,  -55.09071827, -227.10699379,
         468.47905737,   56.87455249,  166.57849157,  207.8778075 ,
         310.47865587,   14.46751943]))

In [4]:
# using separate instances for each activation works

class WorkingTestNet(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.layer_1 = torch.nn.Linear(input_size, 32)
        self.layer_2 = torch.nn.Linear(32, 32)
        self.layer_3 = torch.nn.Linear(32, 1)
        self.activation_1 = torch.nn.ReLU()
        self.activation_2 = torch.nn.ReLU()


    def forward(self, x):
        h = self.activation_1(self.layer_1(x))
        h = self.activation_2(self.layer_2(h))
        z = self.layer_3(h)
        return z


test_net = WorkingTestNet()

loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(test_net.parameters(), lr=0.001)

for k in range(5000):
    optimizer.zero_grad()
    y_pred = test_net(X_train)
    loss_value = loss_function(y_pred, y_train)
    loss_value.backward()
    optimizer.step()
    
e = shap.DeepExplainer(test_net, X_train)
shap_values = e.shap_values(X_test)

test_net(X_test)[:10, 0], e.expected_value + np.sum(shap_values, axis=1)[:10]

(tensor([ 231.5638,   40.2309,  -66.5759, -137.0923,  216.4595,   31.5131,
          208.5716,  131.9723,  181.8214,   28.3996], grad_fn=<SelectBackward>),
 array([ 231.56376983,   40.23091358,  -66.57589126, -137.09228539,
         216.45952581,   31.51306009,  208.57163727,  131.97229829,
         181.82143238,   28.39964405]))